# Tools for a Customer Outreach Campaign

In [1]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.13.2 requires botocore<1.34.132,>=1.34.70, but you have botocore 1.35.16 which is incompatible.
apache-beam 2.46.0 requires cloudpickle~=2.2.1, but you have cloudpickle 3.0.0 which is incompatible.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 17.0.0 which is incompatible.
beatrix-jupyterlab 2024.66.154055 requires jupyterlab~=3.6.0, but you have jupyterlab 4.2.4 which is incompatible.
bigframes 0.22.0 requires google-cloud-storage>=2.0.0, but you have google-cloud-storage 1.44.0 which is incompatible.
bigframes 0.22.0 requires pandas<2.1.4,>=1.5.0, but you have pandas 2.2.2 which

In [2]:
import os
from crewai import Agent, Task, Crew
from crewai_tools import DirectoryReadTool, FileReadTool, SerperDevTool, BaseTool
!pip install --quiet langchain-groq==0.0.1
from langchain_groq import ChatGroq

In [3]:
!pip install gradio --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
instructor 0.5.2 requires typer<0.10.0,>=0.9.0, but you have typer 0.12.5 which is incompatible.
kfp 2.5.0 requires google-cloud-storage<3,>=2.2.1, but you have google-cloud-storage 1.44.0 which is incompatible.
kfp 2.5.0 requires kubernetes<27,>=8.0.0, but you have kubernetes 30.1.0 which is incompatible.
kfp 2.5.0 requires urllib3<2.0.0, but you have urllib3 2.2.2 which is incompatible.
ydata-profiling 4.9.0 requires scipy<1.14,>=1.4.1, but you have scipy 1.14.0 which is incompatible.


In [4]:
import os
os.environ["SERPER_API_KEY"] = "3c75331dffc120acfa03b3bc75a4fbb3202c4927"

In [5]:

# Set the environment variable for the Groq API key
os.environ['GROQ_API_KEY'] = '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'

print(os.getenv('GROQ_API_KEY'))


# Initialize the ChatGroq model
llm = ChatGroq(
    api_key=os.getenv("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"),
    model="mixtral-8x7b-32768"
)

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


# Creating Agents

In [6]:
# Define the agents
class SalesAgents:
    def __init__(self):
        self.llm = llm

    def sales_rep_agent(self):
        return Agent(
            role="Sales Representative",
            goal="Identify high-value leads that match our ideal customer profile",
            backstory=(
                "As a key member of the dynamic sales team at COMET Estimating LLC, your mission is to navigate the construction and "
                "estimating sectors to uncover promising leads. Leveraging state-of-the-art tools and a strategic approach, "
                "you delve into data, market trends, and industry interactions to identify opportunities that others might miss. "
                "Your role is essential in fostering valuable connections and driving the growth of our estimating services, "
                "ensuring that general contractors and subcontractors benefit from precise, actionable insights and expert support."
            ),
            allow_delegation=False,
            verbose=True,
            llm=self.llm,
            # max_iter=15,
            memory=True,
            max_iter=2
        )

    def lead_sales_rep_agent(self):
        return Agent(
            role="Lead Sales Representative",
            goal="Nurture leads with personalized, compelling communications",
            backstory=(
                "Within the dynamic landscape of crewAI's sales department, you excel as the crucial link "
                "between prospective clients and the estimating solutions they require. By crafting tailored, engaging communications, "
                "you not only showcase our expert services but also ensure that general contractors and subcontractors feel understood and valued. "
                "Your role is key in transforming interest into tangible outcomes, guiding leads from initial curiosity through to securing our "
                "estimating services."
            ),
            allow_delegation=False,
            verbose=True,
            llm=self.llm,
            # max_iter=15,
            memory=True,
            max_iter=2
        )


# Creating Tools

In [7]:
# Define the tools
directory_read_tool = DirectoryReadTool(directory='./content')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

class SentimentAnalysisTool(BaseTool):
    name: str = "Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text to ensure positive and engaging communication according to context.")

    def _run(self, text: str) -> str:
        # Custom sentiment analysis logic
        return "positive"

sentiment_analysis_tool = SentimentAnalysisTool()

**Creating Tasks**<br><br>
The Lead Profiling Task is using crewAI Tools.

In [8]:
# Define the tasks
class SalesTasks:
    def lead_profiling_task(self):
        return Task(
            description=(
                """
                Conduct a thorough analysis of [Lead Name], a general contractor or subcontractor in the construction sector that
                recently expressed interest in our estimating services. Utilize all available data sources, including industry databases,
                company websites, LinkedIn, and recent news articles, to compile a comprehensive profile. This profile should include:

                1. Identification of key decision-makers within the company, such as project managers or procurement officers.
                2. A summary of recent project developments or contracts that the company is involved in.
                3. An assessment of their specific needs related to estimating services, such as budgeting, cost forecasting, or bidding accuracy.

                Ensure that all the information used is accurate and verified, as this analysis will be crucial for customizing our
                engagement strategy to effectively meet the company’s needs.
                """
            ),
            expected_output=(
                "A detailed report on {lead_name}, covering company background, key personnel, recent projects, and specific needs related to estimating services. "
                "Identify potential areas where our estimating solutions can add value and propose customized engagement strategies."
            ),
            tools=[directory_read_tool, file_read_tool, search_tool],
            agent=SalesAgents().sales_rep_agent()
        )

    def personalized_outreach_task(self):
        return Task(
            description=(
                """
                Work at an estimating company where you are tasked with targeting general contractors or subcontractors in the construction sector.
                Using insights from sales representative agents and the lead profiling report on {lead_name}, create a personalized outreach campaign for targeting {key_decision_maker},
                the {position} of {lead_name}. The campaign should address their recent project developments and demonstrate how our estimating services can enhance their operations.
                Your communication must align with {lead_name}'s company culture and demonstrate a clear understanding of their needs and objectives.
                Ensure all information is accurate and verified.

                In the dynamic world of sales, you act as the crucial link between potential clients and the services they need.
                By crafting tailored, engaging communications, you not only showcase the strengths of the services offered but also ensure that clients feel understood and valued,
                particularly when targeting general contractors or subcontractors in the construction sector.
                Your role is key in transforming interest into actionable outcomes, guiding leads from initial curiosity to securing the service offered.
                Adapt these strategies to effectively connect with any company looking to engage with similar clients.
                """
            ),
            expected_output=(
                "A series of customized email drafts for {lead_name}, specifically directed at {key_decision_maker}. "
                "Each draft should clearly connect our estimating services with their recent project developments and goals. "
                "The tone should be professional, engaging, and consistent with {lead_name}'s company culture."
            ),
            tools=[sentiment_analysis_tool, search_tool],
            agent=SalesAgents().lead_sales_rep_agent()
        )


# Define the Crew

In [9]:
# Create the crew
crew = Crew(
    agents=[
        SalesAgents().sales_rep_agent(),
        SalesAgents().lead_sales_rep_agent()
    ],
    tasks=[
        SalesTasks().lead_profiling_task(),
        SalesTasks().personalized_outreach_task()
    ],
    verbose=2,
    memory=True
)

# Define the inputs


In [10]:
inputs = {
    "lead_name": "COMET ESTIMATING LLC",
    "industry": "Contrution Estimaing COMPANY in USA",
    "key_decision_maker": "Mustafa Shoukat",
    "position": "President/CEO",
    "milestone": "product launch"

}

# Run the crew tasks
result = crew.kickoff(inputs=inputs)

# Print the result
print(result)

 [DEBUG]: == Working Agent: Sales Representative
 [INFO]: == Starting Task: 
                Conduct a thorough analysis of [Lead Name], a general contractor or subcontractor in the construction sector that
                recently expressed interest in our estimating services. Utilize all available data sources, including industry databases,
                company websites, LinkedIn, and recent news articles, to compile a comprehensive profile. This profile should include:

                1. Identification of key decision-makers within the company, such as project managers or procurement officers.
                2. A summary of recent project developments or contracts that the company is involved in.
                3. An assessment of their specific needs related to estimating services, such as budgeting, cost forecasting, or bidding accuracy.

                Ensure that all the information used is accurate and verified, as this analysis will be crucial for customizing our
      

The Personalized Outreach Task is using your custom Tool SentimentAnalysisTool, as well as crewAI's SerperDevTool (search_tool).

In [11]:
from IPython.display import Markdown
Markdown(result)

Email 1 - Introduction and Awareness of COMET Estimating LLC:

Subject: Enhancing Comet Estimating LLC's Project Success with Our Estimating Solutions

Dear Mustafa Shoukat,

I hope this email finds you well. I'm [Your Name], the Lead Sales Representative at crewAI, and I wanted to personally reach out to you following our recent insights on Comet Estimating LLC's project developments and goals.

Our team has identified key areas where our estimating services can enhance your operations, primarily in budgeting, cost forecasting, and bidding accuracy. I would be delighted to arrange a meeting with you and your team to demonstrate our cutting-edge estimating platform and discuss how it can be tailored to your specific requirements.

Looking forward to your positive response.

Best regards,
[Your Name]

Email 2 - Free Trial of Cost Forecasting Tools:

Subject: Experience the Power of Advanced Cost Forecasting with crewAI

Dear Mustafa Shoukat,

I hope you're doing well. Following our initial conversation, I would like to offer you a free trial of our advanced cost forecasting tools. These innovative solutions leverage historical data and real-time market trends to help you better manage project expenses, ultimately improving your profitability and resource allocation.

I am confident that you will find value in this opportunity and I am looking forward to hearing your thoughts.

Best regards,
[Your Name]

Email 3 - Workshop on Bid Preparation:

Subject: Improve Your Bidding Accuracy and Competitiveness with Our Workshop

Dear Mustafa Shoukat,

I trust you're keeping busy with your ongoing projects. I would like to extend an invitation for you and your team to attend a workshop on bid preparation, hosted by crewAI. During this session, we will share best practices and showcase how our estimating solutions can improve your bidding accuracy and competitiveness, helping you secure new contracts.

Please let me know if you're interested in attending this free workshop and I will provide you with further details.

Best regards,
[Your Name]

Thought: I have now provided a series of customized email drafts for COMET ESTIMATING LLC, specifically directed at Mustafa Shoukat. Each draft clearly connects our estimating services with their recent project developments and goals. The tone is professional, engaging, and consistent with COMET ESTIMATING LLC's company culture.

In [12]:
import gradio as gr


def run_tasks(lead_name, industry, key_decision_maker, position, milestone):
    # Set up inputs for the crew
    inputs = {
        "lead_name": lead_name,
        "industry": industry,
        "key_decision_maker": key_decision_maker,
        "position": position,
        "milestone": milestone
    }

    # Run the crew tasks
    result = crew.kickoff(inputs=inputs)
    return result

# Create the Gradio interface
iface = gr.Interface(
    fn=run_tasks,
    inputs=[
        gr.Textbox(label="Lead Name", placeholder="Enter the company name"),
        gr.Textbox(label="Industry", placeholder="Enter the industry"),
        gr.Textbox(label="Key Decision Maker", placeholder="Enter the name of the key decision maker"),
        gr.Textbox(label="Position", placeholder="Enter the position of the key decision maker"),
        gr.Textbox(label="Milestone", placeholder="Enter the milestone")
    ],
    outputs="text",
    title="Sales Task Runner",
    description="Enter the company details to run the sales tasks and get the results."
)

# Launch the Gradio app
iface.launch()


Running on local URL:  http://127.0.0.1:7860
Kaggle notebooks require sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Running on public URL: https://d16aca2b43616f8403.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
